In [1]:
# Loading the cv libraries 
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 

# Example pipeline for snow detection

In [2]:
# Defining the video path 
video_dir = 'videos'
video_to_use = 'DJI_0212.MP4'
path_to_video = os.path.join(video_dir, video_to_use)

# Defining the path to the images dir where the video will be split
images_dir = 'images'
images_dir_video = os.path.join(images_dir, video_to_use.split('.')[0])

# Creating the images dir if it does not exist
if not os.path.exists(images_dir):
    os.mkdir(images_dir)
if not os.path.exists(images_dir_video):
    os.mkdir(images_dir_video)

# Opening the video
cap = cv2.VideoCapture(path_to_video)

# Getting the total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Getting the frame rate
fps = cap.get(cv2.CAP_PROP_FPS)

# Getting the frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Getting the frame height
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Defining the top frames to use 
top_frames = 1000

# Deleting the images dir if it exists
if os.path.exists(images_dir_video):
    for file in os.listdir(images_dir_video):
        os.remove(os.path.join(images_dir_video, file))

# Spliting the video into frames
for i in tqdm(range(top_frames)):
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(os.path.join(images_dir_video, f'{i}.jpg'), frame)
    else:
        break

100%|██████████| 1000/1000 [00:36<00:00, 27.44it/s]


In [3]:
# Defining the buffer of pixels to crop around the center 
buffer = 10

# Running all the images in the images dir

In [4]:
# Creating the output dir 
cropped_image_dir = 'cropped_images'
if not os.path.exists(cropped_image_dir):
    os.makedirs(cropped_image_dir)

cropped_image_dir_video = os.path.join(cropped_image_dir, video_to_use.split('.')[0])
if not os.path.exists(cropped_image_dir_video):
    os.makedirs(cropped_image_dir_video)

# Removing the images in the cropped image dir if it exists
if os.path.exists(cropped_image_dir_video):
    for file in os.listdir(cropped_image_dir_video):
        os.remove(os.path.join(cropped_image_dir_video, file))

# Defining the function that returns the color for a given gray value
def get_color(gray_value):
    if gray_value < 100:
        return 'green'
    else:
        return 'red'

# Sorting the images 
images_names = os.listdir(images_dir_video)
images_names.sort(key=lambda x: int(x.split('.')[0]))

# Iterating over the images in the images dir and cropping them 
for image_name in tqdm(images_names):
    # Saving the path to the image 
    image_path = os.path.join(images_dir_video, image_name)
    
    # Loading the image in the RGB format
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    # Extracting the center point
    h_center, w_center = int(image.shape[0]/2), int(image.shape[1]/2)
    
    # Calculating the top left and bottom right coordinates
    top_left = (w_center - buffer, h_center - buffer)
    bottom_right = (w_center + buffer, h_center + buffer)
    
    # Cropping the image
    cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    
    # Calculating the mean of the cropped image
    gray_value = cropped_image.mean()

    # Saving the cropped image
    cropped_image_path = os.path.join(cropped_image_dir_video, image_name)

    # Creating a fig object with two axes: 
    # On the left there will be the original image with the rectangle
    # On the right there will be the cropped image with the mean gray value
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(image_name)

    # Ploting the original image with the rectangle
    axes[0].imshow(image)
    axes[0].set_title('Original image')
    axes[0].axis('off')
    axes[0].add_patch(plt.Rectangle(top_left, bottom_right[0] - top_left[0], bottom_right[1] - top_left[1], fill=False, color=get_color(gray_value)))

    # Ploting the cropped image with the mean gray value
    axes[1].imshow(cropped_image, cmap='gray')
    axes[1].set_title('Mean gray value: {}'.format(round(gray_value, 2)))

    # Saving the ploted image
    plt.savefig(cropped_image_path)

    # Closing the fig object to free up memory
    plt.close(fig)

100%|██████████| 1000/1000 [03:44<00:00,  4.46it/s]


# Making a video 

In [5]:
# Listing all the cropped images for the current dir 
cropped_images = os.listdir(cropped_image_dir_video)

# Sorting the images by name
cropped_images.sort(key=lambda x: int(x.split('.')[0]))

# Creating a video writer object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# Creating the output_videos dir 
output_videos_dir = 'output_videos'
if not os.path.exists(output_videos_dir):
    os.mkdir(output_videos_dir)

# Defining the path to the specific output video
output_video_path = os.path.join(output_videos_dir, video_to_use)

# Reading the first image to get the size
first_image_path = os.path.join(cropped_image_dir_video, cropped_images[0])

# Reading the first image
first_image = cv2.imread(first_image_path)

# Getting the size of the first image
size = (first_image.shape[1], first_image.shape[0])

fps = 24

# Creating the video writer object
out = cv2.VideoWriter(output_video_path, fourcc, fps, size)

# Iterating over the cropped images and writing them to the video
for cropped_image in tqdm(cropped_images):
    # Reading the image
    image_path = os.path.join(cropped_image_dir_video, cropped_image)
    image = cv2.imread(image_path)

    # Writing the image to the video
    out.write(image)

cv2.destroyAllWindows()

# Closing the video writer
out.release()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:08<00:00, 118.69it/s]
